In [7]:
import pandas as pd
from sklearn import datasets
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import mlflow
from mlflow.models import infer_signature

In [8]:
## set the tracking uri
mlflow.set_tracking_uri(uri="http://127.0.0.1:5000")

In [9]:
## load the dataset
X,y = datasets.load_iris(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2)

# Define the model hyperparameter
params = {"penalty":"l2", "solver":"lbfgs", "max_iter":1000, "multi_class":"auto", "random_state":8888}

lr = LogisticRegression(**params)
lr.fit(X_train, y_train)

d:\data_science\MLFlow\venv\Lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


LogisticRegression(max_iter=1000, multi_class='auto', random_state=8888)

In [10]:
# Prediction on the test set
y_pred=lr.predict(X_test)
y_pred

array([1, 0, 0, 2, 0, 0, 2, 1, 0, 0, 2, 2, 0, 0, 0, 2, 2, 2, 1, 1, 1, 0,
       2, 2, 1, 1, 2, 1, 2, 1])

In [11]:
accuracy = accuracy_score(y_test, y_pred)
print(accuracy)

0.9666666666666667


In [12]:
## MLflow tracking
mlflow.set_tracking_uri(uri="http://127.0.0.1:5000")

mlflow.set_experiment("MLFLOW Quickstart")

# start and mlflow run
with mlflow.start_run():
    ## log the hyperparameterization
    mlflow.log_params(params)   
    
    # log accuracy metrics
    mlflow.log_metric("accuracy", accuracy)
    
    # set a tag
    mlflow.set_tag("Training info", "Basic LR model for iris data")
    
    # infer the model signature
    signature=infer_signature(X_train, lr.predict(X_train)) 
    
    # log the model
    model_info = mlflow.sklearn.log_model(
        sk_model=lr,
        artifact_path="iris_model",
        signature=signature,
        input_example=X_train,
        registered_model_name="trackingg-quickstart",
    )                   

Registered model 'trackingg-quickstart' already exists. Creating a new version of this model...
2024/11/14 22:34:30 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: trackingg-quickstart, version 2
Created version '2' of model 'trackingg-quickstart'.
2024/11/14 22:34:32 INFO mlflow.tracking._tracking_service.client: 🏃 View run peaceful-finch-34 at: http://127.0.0.1:5000/#/experiments/451536116134475627/runs/f75ad7202be64e63a9283fa7fb66b81f.
2024/11/14 22:34:32 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5000/#/experiments/451536116134475627.


In [13]:
# Define the model hyperparameters
params = {"solver": "newton-cg", "max_iter": 1000, "multi_class": "auto", "random_state": 1000}

##train the model

lr=LogisticRegression(**params)
lr.fit(X_train,y_train)
y_pred=lr.predict(X_test)
accuracy=accuracy_score(y_test,y_pred)
print(accuracy)

0.9666666666666667


d:\data_science\MLFlow\venv\Lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


In [14]:
## Sstart an MLFLOW run
with mlflow.start_run():
    ## log the hyperparameters
    mlflow.log_params(params)

    ## Log the accuracy metrics
    mlflow.log_metric("accuracy",accuracy)

    # Set a tag that we can use to remind ourselves what this run was for
    mlflow.set_tag("Training Info", "Basic LR model for iris data")

    ## Infer the model signature

    signature=infer_signature(X_train,lr.predict(X_train))

    ## log the model
    model_info=mlflow.sklearn.log_model(
        sk_model=lr,
        artifact_path="iris_model",
        signature=signature,
        input_example=X_train,
        registered_model_name="tracking-quickstart",

    )

Registered model 'tracking-quickstart' already exists. Creating a new version of this model...
2024/11/14 22:34:39 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: tracking-quickstart, version 2
Created version '2' of model 'tracking-quickstart'.
2024/11/14 22:34:40 INFO mlflow.tracking._tracking_service.client: 🏃 View run nebulous-wasp-506 at: http://127.0.0.1:5000/#/experiments/451536116134475627/runs/6232fce5329a4143a75c1ec1af4ab9c0.
2024/11/14 22:34:40 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5000/#/experiments/451536116134475627.


In [15]:
model_info.model_uri

'runs:/6232fce5329a4143a75c1ec1af4ab9c0/iris_model'

## Load the model back for prediction as a generic python function model

In [16]:
loaded_model = mlflow.pyfunc.load_model(model_info.model_uri)
predictions=loaded_model.predict(X_test)

iris_features_name=datasets.load_iris().feature_names
result=pd.DataFrame(X_test,columns=iris_features_name)
result["actual_class"]=y_test
result["predicted_class"]=predictions

In [17]:
result.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),actual_class,predicted_class
0,5.6,2.7,4.2,1.3,1,1
1,5.1,3.7,1.5,0.4,0,0
2,4.8,3.0,1.4,0.1,0,0
3,6.8,3.2,5.9,2.3,2,2
4,5.1,3.8,1.6,0.2,0,0


## Model registery

In [18]:
## log the model
model_info=mlflow.sklearn.log_model(
    sk_model=lr,
    artifact_path="iris_model",
    signature=signature,
    input_example=X_train,
    registered_model_name="tracking-quickstart",

)

Registered model 'tracking-quickstart' already exists. Creating a new version of this model...
2024/11/14 22:34:46 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: tracking-quickstart, version 3
Created version '3' of model 'tracking-quickstart'.


In [19]:
##create a new MLFLOW experiment
mlflow.set_experiment("MLFLOW Quickstart")

## Sstart an MLFLOW run
with mlflow.start_run():
    ## log the hyperparameters
    mlflow.log_params(params)

    ## Log the accuracy metrics
    mlflow.log_metric("accuracy",1.0)

    # Set a tag that we can use to remind ourselves what this run was for
    mlflow.set_tag("Training Info2", "Basic LR model for iris data")

    ## Infer the model signature

    signature=infer_signature(X_train,lr.predict(X_train))

    ## log the model
    ## log the model
    model_info=mlflow.sklearn.log_model(
        sk_model=lr,
        artifact_path="iris_model",
        signature=signature,
        input_example=X_train,

    )

Exception: Run with UUID 42ed68fb3afa4acfa98aaab4ffb8f0ec is already active. To start a new run, first end the current run with mlflow.end_run(). To start a nested run, call start_run with nested=True